In [50]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [3]:
import geopandas as gpd 
import folium
from geopy.geocoders import Nominatim
import folium.plugins
import branca
import branca.colormap as cm
from folium.plugins import MarkerCluster

In [11]:
DATA_PATH = Path('../../../data/preprocessed/safegraph/social-distancing/FIPS/450/2020')

In [61]:
county_fips = '45039'
state_fips = '45'

In [13]:
month = '06'
day = '01'

df = pd.read_csv(DATA_PATH / f'{month}/{day}/2020-{month}-{day}-social-distancing.csv')

In [59]:
df['county_fips'] = df['origin_census_block_group'].apply(lambda x: str(x)[:5])
df['state_fips'] = df['origin_census_block_group'].apply(lambda x: str(x)[:2])

In [60]:
df.head()

,origin_census_block_group,date_range_start,date_range_end,device_count,distance_traveled_from_home,bucketed_distance_traveled,median_dwell_at_bucketed_distance_traveled,completely_home_device_count,median_home_dwell_time,bucketed_home_dwell_time,...,median_non_home_dwell_time,candidate_device_count,bucketed_away_from_home_time,median_percentage_time_home,bucketed_percentage_time_home,mean_home_dwell_time,mean_non_home_dwell_time,mean_distance_traveled_from_home,county_fips,state_fips
0,450410007002,2020-06-01T00:00:00-04:00,2020-06-02T00:00:00-04:00,65,1160,"{""16001-50000"":2,""0"":26,"">50000"":7,""2001-8000""...","{""16001-50000"":33,"">50000"":187,""<1000"":35,""200...",26,598,"{""721-1080"":2,""361-720"":12,""61-360"":10,""<60"":1...",...,16,152,"{""21-45"":4,""541-600"":1,""46-60"":1,""721-840"":3,""...",95,"{""0-25"":16,""76-100"":40,""51-75"":5,""26-50"":4}",629,226,2134,45041,45
1,450450028053,2020-06-01T00:00:00-04:00,2020-06-02T00:00:00-04:00,141,3402,"{""16001-50000"":5,""0"":26,"">50000"":17,""2001-8000...","{""16001-50000"":102,"">50000"":83,""<1000"":71,""200...",25,579,"{""721-1080"":26,""361-720"":17,""61-360"":12,""<60"":...",...,209,282,"{""21-45"":10,""481-540"":2,""541-600"":6,""46-60"":4,...",67,"{""0-25"":50,""76-100"":57,""51-75"":19,""26-50"":6}",601,350,20900,45045,45
2,450630210092,2020-06-01T00:00:00-04:00,2020-06-02T00:00:00-04:00,821,4695,"{""16001-50000"":187,""0"":192,"">50000"":53,""2001-8...","{""16001-50000"":49,"">50000"":47,""<1000"":103,""200...",192,862,"{""721-1080"":201,""361-720"":117,""61-360"":70,""<60...",...,117,1408,"{""21-45"":52,""481-540"":41,""541-600"":33,""46-60"":...",86,"{""0-25"":131,""76-100"":476,""51-75"":165,""26-50"":45}",803,267,8407,45063,45
3,450830231012,2020-06-01T00:00:00-04:00,2020-06-02T00:00:00-04:00,118,3077,"{""16001-50000"":17,""0"":29,"">50000"":5,""2001-8000...","{""16001-50000"":47,"">50000"":99,""<1000"":158,""200...",28,622,"{""721-1080"":25,""361-720"":9,""61-360"":12,""<60"":3...",...,133,261,"{""21-45"":5,""481-540"":12,""541-600"":2,""46-60"":7,...",70,"{""0-25"":36,""76-100"":53,""51-75"":21,""26-50"":3}",620,301,4906,45083,45
4,450190049012,2020-06-01T00:00:00-04:00,2020-06-02T00:00:00-04:00,63,802,"{""16001-50000"":3,""0"":16,"">50000"":10,""2001-8000...","{""16001-50000"":310,"">50000"":70,""<1000"":145,""20...",18,553,"{""721-1080"":11,""361-720"":7,""61-360"":6,""<60"":21...",...,125,121,"{""21-45"":3,""481-540"":2,""541-600"":1,""46-60"":2,""...",78,"{""0-25"":18,""76-100"":32,""51-75"":6,""26-50"":1}",619,371,2573,45019,45


In [62]:
df2 = df[df['county_fips'] == county_fips]
df3 = df[df['state_fips'] == state_fips]

3044

In [103]:
def transition_matrix(df):

    census_block_group_ids = df['origin_census_block_group'].unique()
    cbg_idx = {cbg_id: idx for idx, cbg_id in enumerate(census_block_group_ids)}

    num_block_groups = len(census_block_group_ids)

    travel_amounts = np.zeros((num_block_groups, num_block_groups))

    for i in range(len(df)):
        row = df.iloc[i]

        origin_id = row['origin_census_block_group']
        if origin_id in cbg_idx:
            origin_idx = cbg_idx[origin_id]
        else:
            continue
        #if i < 1:
        #    print(f'origin_id: {origin_id}')

        dests = json.loads(row['destination_cbgs'])
        #if i < 1:
        #    print(dests)
        for dest_id, num_devices in dests.items():
            dest_id = int(dest_id)
            if dest_id in cbg_idx:
                #if i < 10:
                #    print(f'{origin_id} traveled to {dest_id}: {num_devices} devices')    
                dest_idx = cbg_idx[dest_id]
                travel_amounts[dest_idx][origin_idx] += num_devices

    return travel_amounts

## Travel Within the State

In [104]:
travel_amounts = transition_matrix(df3)
travel_amounts

array([[ 52.,   0.,   0., ...,   0.,   0.,   0.],
       [  0., 107.,   0., ...,   0.,   1.,   0.],
       [  0.,   0., 743., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  39.,   0.,   0.],
       [  0.,   0.,   0., ...,   0., 138.,   0.],
       [  0.,   0.,   2., ...,   0.,   0., 110.]])

In [105]:
# Number of entries filled:
len(travel_amounts.nonzero()[0])

210350

In [106]:
# Number of block groups:
num_block_groups = travel_amounts.shape[0]
num_block_groups

3044

In [107]:
# Portion of the matrix that's filled:
len(travel_amounts.nonzero()[0]) / (num_block_groups * num_block_groups)

0.02270143027104871

## Travel Within the County

In [108]:
travel_amounts = transition_matrix(df2)
travel_amounts

array([[ 78.,   9.,   3.,   1.,   0.,   3.,   0.,   7.,   0.,   0.,   2.,
          1.,   1.,   4.,   7.,   1.,   4.,   2.],
       [  3.,  86.,   0.,   0.,   0.,   1.,   0.,   0.,   1.,   0.,   2.,
          2.,   0.,   3.,   3.,   0.,   0.,   2.],
       [ 13.,   1.,  59.,   0.,   0.,   1.,   0.,   5.,   1.,   0.,   4.,
          0.,   1.,   3.,   5.,   0.,   4.,   3.],
       [  0.,   0.,   1.,  39.,   0.,   0.,   0.,   0.,   1.,   1.,   0.,
          0.,   3.,   0.,   1.,   0.,   0.,   0.],
       [  0.,   0.,   1.,   1.,  69.,   1.,   1.,   0.,   0.,   1.,   0.,
          1.,   0.,   0.,   1.,   1.,   0.,   0.],
       [  3.,   2.,   1.,   0.,   0.,  25.,   0.,   1.,   0.,   2.,   1.,
          3.,   0.,   2.,   3.,   3.,   1.,   0.],
       [  1.,   0.,   0.,   0.,   2.,   0.,  97.,   0.,   0.,   4.,   0.,
          2.,   0.,   1.,   0.,   0.,   0.,   1.],
       [  4.,   1.,   6.,   0.,   0.,   0.,   0.,  50.,   0.,   0.,   4.,
          3.,   1.,   2.,   0.,   2.,   3.,   3.],


In [109]:
# Number of entries filled:
len(travel_amounts.nonzero()[0])

193

In [110]:
# Number of block groups:
num_block_groups = travel_amounts.shape[0]
num_block_groups

18

In [111]:
# Portion of the matrix that's filled:
len(travel_amounts.nonzero()[0]) / (num_block_groups * num_block_groups)

0.595679012345679

In [113]:
eigenvalues, eigenvectors = np.linalg.eig(travel_amounts)

In [114]:
eigenvalues

array([120.26258828+0.j        , 103.83280023+0.j        ,
        96.31980653+0.j        ,  83.13093267+0.j        ,
        24.83652457+0.j        ,  69.10655686+0.j        ,
        67.04927709+0.j        ,  65.56275969+0.j        ,
        61.62979762+0.j        ,  30.9300983 +0.j        ,
        31.90058593+0.j        ,  34.13145462+0.j        ,
        54.60478538+0.j        ,  53.13661963+0.j        ,
        49.08183571+1.31774617j,  49.08183571-1.31774617j,
        42.08696429+0.j        ,  43.31477688+0.j        ])

In [122]:
largest_cbg_idx = eigenvectors[0].argmax()

In [123]:
df['origin_census_block_group'].unique()[largest_cbg_idx]

450450028053